In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import sklearn
import os
import os
import numpy as np
import concurrent.futures
from skimage.feature import hog
from skimage import exposure
from skimage.feature import local_binary_pattern
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from PIL import Image
import sys
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.gridspec import GridSpec

os.environ['NOVA_HOME'] = '/home/labs/hornsteinlab/Collaboration/NOVA_GAL/NOVA'

sys.path.insert(1, os.getenv('NOVA_HOME'))
sys.path.insert(1, os.getenv("NOVA_HOME"))
print(f"NOVA_HOME: {os.getenv('NOVA_HOME')}")

from src.preprocessing.preprocessing_utils import get_image_focus_quality 
from src.preprocessing.preprocessing_utils import rescale_intensity, fit_image_shape

%reload_ext autoreload
%autoreload 2
%aimport

%matplotlib inline

NOVA_HOME: /home/labs/hornsteinlab/Collaboration/NOVA_GAL/NOVA
Modules to reload:
all-except-skipped

Modules to skip:



# Utils

In [2]:
# Load images
def get_npy_files(path):
    npy_files = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.npy'):
                npy_files.append(os.path.join(root, file))
    return npy_files

import skimage


def get_metrics_processed(tile):
  metrics = ()
  for c in range(tile.shape[-1]):
    metrics += get_metrics(tile[...,c])
    
  return metrics

def get_metrics(tile, as_string=False):
    sharpness_brenner = get_image_focus_quality(tile)    
    if as_string:
        return f"Brenner: {round(sharpness_brenner, 3)}"
    return sharpness_brenner

# def get_metrics_columns():
#   return ["Path", "Quality", "Target_SNR", "Target_Sharpness_Laplacian", "Target_Var",
#            'Target_Sharpness_Brenner', "Target_Entropy", "Target_Sigma", "Target_HighFreq", "DAPI_SNR", "DAPI_Sharpness_Laplacian",
#             "DAPI_Var", 'DAPI_Sharpness_Brenner', "DAPI_Entropy", "DAPI_Sigma", "DAPI_HighFreq"]


def run_dim_reduction(dim_red, images, labels=None, show=True):
  # Perform PCA
  images = images.reshape(images.shape[0], -1)
  reduced = dim_red.fit_transform(images)
  
  if not show:
    return reduced
  
  if labels is None:
    plt.scatter(reduced[:,0], reduced[:,1])
    plt.show()
    return reduced
  
  
  labels_unique = np.unique(labels)
  
  for l in labels_unique:
    indexes = labels == l
    plt.scatter(reduced[indexes,0], reduced[indexes,1], alpha=0.5)
  plt.legend(labels_unique)
  plt.show()
  
  return reduced 

def plot_images(images, vmin=0,vmax=1000):
    for i in range(len(images)):
        fig, ax = plt.subplots(1, 2)
        ax[0].imshow(images[i,...,0])#, vmin=vmin,vmax=vmax)
        ax[0].set_title("Target")
        ax[1].imshow(images[i,...,1])#, vmin=vmin,vmax=vmax)
        ax[1].set_title("Nucleus")
        plt.show()
        
def load_tiles(paths):
  paths_split = map(lambda x: (x.rsplit('_',1)[0], int(x.rsplit('_',1)[1])), paths.reshape(-1,))
  tiles = []
  for filename, tile_number in paths_split:
    t = np.load(filename)[tile_number, ...]
    tiles.append(t)
    
  tiles = np.stack(tiles)
  return tiles



def get_outliers(df, feature, split=False, iqrs=1.5):
    # Calculate Q1, Q3, and IQR
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    
    lower = df[feature] < Q1 - iqrs* IQR
    higher = df[feature] > Q3 + iqrs*IQR
    if split:
        # Determine outliers using IQR
        outliers_lower = df[lower]
        outliers_higher = df[higher]
        return outliers_lower, outliers_higher
    
    # Determine outliers using IQR
    outliers = df[(lower ) | (higher)]
    
    return outliers

def show_images(df, max_samples = 10):
    for ind, path in enumerate(df.Path.values):
        print(ind)
        if max_samples is not None and ind >= max_samples:
            print(f"Stopping at {ind}. There are {len(df.Path.values)} images in total")
            break
        
        # Target
        target_path = os.path.join(d, path)
        show_processed_tif(target_path)
        # His DAPI
        # path_l = target_path.split("/")
        # path_l[-2] = 'DAPI'
        
        # file_name = path_l[-1].split("_")
        # dapi_file_name = "_".join([file_name[0], 'w1confDAPI', file_name[-1]])
        # dapi_file_name = "/".join([*path_l[:-1], dapi_file_name])
        # print(dapi_file_name)

        # show_processed_tif(dapi_file_name)
        print('--------------------------------')
        
def get_dapi_file_name(path):
    site_path, tile_number = path.rsplit('_',1)
    path_l = site_path.split("/")
    path_l[-2] = 'DAPI'

    file_name = path_l[-1].split("_")
    dapi_file_name = "_".join([file_name[0], 'w1confDAPI', file_name[-1]])
    dapi_file_name = "/".join([*path_l[:-1], f'{dapi_file_name}_{tile_number}'])
    
    return dapi_file_name
        
def show_tiles(df, max_samples=10, rescale_tile=False):
    for ind, path in enumerate(df.Path.values):
        site_path, tile_number = path.rsplit('_',1)
        tile_number = int(tile_number)
        print(ind)
        if max_samples is not None and ind >= max_samples:
            print(f"Stopping at {ind}. There are {len(df.Path.values)} images in total")
            break
        
        # Target
        show_tile(site_path, tile_number, rescale_tile)
        # His DAPI
        path_l = site_path.split("/")
        path_l[-2] = 'DAPI'
        
        file_name = path_l[-1].split("_")
        dapi_file_name = "_".join([file_name[0], 'w1confDAPI', file_name[-1]])
        dapi_file_name = "/".join([*path_l[:-1], dapi_file_name])
        print(dapi_file_name)

        show_tile(dapi_file_name, tile_number, rescale_tile)
        print('--------------------------------')
        
# def save_to_mapping(mappings, marker, metric_name, low_threshold, high_threshold):
#     mappings[marker] = {
#         metric_name: {
#             'low_threshold': low_threshold,
#             'high_threshold': high_threshold
#         }
#     }

def init_mappings(markers=[], filepath=None):
    if filepath is not None:     
        if os.path.exists(filepath):
            mappings = pd.read_csv(filepath, index_col=0)
            return mappings
        
    mappings = pd.DataFrame(columns=['Lower_bound', 'Upper_bound'], index=markers)

    return mappings
        
def save_to_mapping(filepath, mappings, marker, value, is_upper_bound):
    col = 'Upper_bound' if is_upper_bound else 'Lower_bound' 
    mappings.loc[marker, col] = value
    
    mappings.to_csv(filepath)
    print(f"File saved to {filepath}")
    
    
def processed_path_to_raw_paths(path):
    path_splited = path.split(os.sep)
    filename = path_splited[-1]
    filename_split = filename.split('_')
    rep = filename_split[0]
    panel = filename_split[4]
    line = filename_split[5]
    filename_new = '_'.join(filename_split[1:4])
    batch = path_splited[-5]
    batch = batch.split('_')[0]
    cond = path_splited[-3]
    marker = path_splited[-2]
    
    ret = os.path.join("/home/labs/hornsteinlab/Collaboration/MOmaps/input/images/raw/Cory/indi-image-pilot-20241128", batch, line, panel, cond, rep, marker, f'{filename_new}.tif')
    return ret

def raw_path_to_processed_path(path):
    path_splited = path.split(os.sep)
    filename = os.path.splitext(path_splited[-1])[0]
    rep = path_splited[-3]
    panel = path_splited[-5]
    line = path_splited[-6]
    batch = path_splited[-7]
    batch = f"{batch}_16bit_no_downsample"
    cond = path_splited[-4]
    marker = path_splited[-2]
    
    ret = os.path.join("/home/labs/hornsteinlab/Collaboration/MOmaps/input/images/processed/spd2/SpinningDisk/", batch, line, cond, marker, f'{rep}_{filename}_{panel}_{line}_processed.npy')
    return ret


d = '/home/labs/hornsteinlab/Collaboration/MOmaps/input/images/raw/Cory/indi-image-pilot-20241128/'

def show_label(path):
    path_l = path.split("/")
    return path_l[-7:]
    
def show_processed_tif(path):
    # read the image stack
    img = cv2.imread(path, cv2.IMREAD_ANYDEPTH) 
    img = fit_image_shape(img, (1024, 1024))
    # rescale pixel intensities
    img = rescale_intensity(img)
    
    # show the image with grid 
    fig, ax = plt.subplots(figsize=(7,7))
    plt.imshow(img, cmap='gray')
    put_tiles_grid(image=img, ax=ax)
    plt.axis('off')
    plt.title(show_label(path), color='purple')
    print(get_metrics(img, True))
    print(f"Img shape: {img.shape}")
    plt.show()
    
def show_tile(site_path, tile_number, rescale_tile=False):
    # read the image stack
    img = cv2.imread(site_path, cv2.IMREAD_ANYDEPTH) 
    img = fit_image_shape(img, (1024, 1024))
    # rescale pixel intensities
    img = rescale_intensity(img)
    
    row_ind = tile_number // 10
    col_ind = tile_number % 10
    img = img[row_ind * 100 : (row_ind + 1) * 100, col_ind * 100 : (col_ind + 1) * 100]
    
    print(f"img shape: {np.asarray(img).shape}")
    
#     # Check dead cells?
#     img_uint8 = img.astype(np.uint8)
#     # Apply Gaussian blur
# #     gray_blurred = cv2.GaussianBlur(img_uint8, (9, 9), 0)

#     # Apply Hough Circle Transform
#     # Adjust the parameters, especially minRadius and maxRadius, to detect small circles
#     circles = cv2.HoughCircles(img_uint8, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=30, minRadius=0, maxRadius=0)
   
    
    if rescale_tile:
        print("NOTICE! Rescaling also the tile!!!!!")
        img = rescale_intensity(img)
    
    fig, ax = plt.subplots(figsize=(7,7))
    plt.imshow(img, cmap='gray', vmin=0, vmax=1)
    
#     if circles is not None:
#         print(f"circles count: {len(circles)}")
#         circles = np.uint16(np.around(circles))
#         for i in circles[0, :]:
#             center = (i[0], i[1])  # circle center
#             radius = i[2]  # circle radius
#             cv2.circle(image, center, radius, (0, 255, 0), 2)
    
#     put_tiles_grid(image=img, ax=ax)
    plt.axis('off')
    plt.title(show_label(site_path), color='purple')
    print(get_metrics(img, True))
    plt.show()
    
def crop_frame(original_image):
    # Crop the image by removing a 12-pixel frame from each side
    cropped_image = original_image[12:1012, 12:1012]  
    return cropped_image

def crop_site_to_tiles(img):
    from skimage.util import view_as_blocks
    # Break the cropped image into 64 tiles of size 100x100
    tile_size = 100
    num_tiles = 64

    # Reshape the image into tiles using view_as_blocks from skimage
    tiles = view_as_blocks(img, block_shape=(tile_size, tile_size, 3))

    for i in range(8):
        for j in range(8):
            tile = tiles[i, j]
    return 

def put_tiles_grid(image, ax):
    # assumes 1000x1000 image
    import matplotlib.patches as patches

    # Add dashed grid lines for 64 blocks
    num_blocks = 10
    block_size = 100

    for i in range(1, num_blocks):
        # Draw horizontal dashed lines
        ax.plot([0, 1000], [i * block_size, i * block_size], linestyle='--', lw=1, alpha=0.5, color='pink')

        # Draw vertical dashed lines
        ax.plot([i * block_size, i * block_size], [0, 1000], linestyle='--', lw=1, alpha=0.5, color='pink')

    # Remove x and y axis labels
    ax.set_xticks([])
    ax.set_yticks([])

    # Add a title
    plt.title('Image with Dashed Grid of 64 Blocks')

def create_histogram(df_marker_brenner, percentiles, low_perc = 0.5, high_perc = 99.9, x_min=None, x_max=None, overlay_Cellline = False):
    hist_range = (percentiles[f'{x_min}%'], percentiles[f'{x_max}%']) if x_min is not None and x_max is not None else None
    plt.hist(df_marker_brenner.values, bins = 100, range=hist_range, color=plt.cm.tab10(range(1))[0], alpha=0.6, label = 'Brenner scores')
    if overlay_Cellline:
        # Overlay histograms for each CellLine
        unique_celllines = df_marker['CellLine'].unique()
        colors = plt.cm.tab10(range(len(unique_celllines)+1))[1:]  # Use a colormap for distinct colors

        for color, cellline in zip(colors, unique_celllines):
            # Filter data for the current CellLine
            cellline_data = df_marker[metric_name][df_marker['CellLine'] == cellline]
            # Plot histogram for the current CellLine with transparency
            plt.hist(cellline_data.values, bins=100, range=hist_range, alpha=0.4, label=f'{cellline}', color=color)

    plt.scatter(percentiles['50%'], 0.5, color='yellow', s=12, label='50th percentile')
    plt.scatter(percentiles[f'{high_perc}%'], 0.5, color='orange', s=12, label=f'{high_perc}th percentile')
    plt.scatter(percentiles[f'{low_perc}%'], 0.5, color='r', s=12, label=f'{low_perc}th percentile')
    plt.legend()
    # plt.show()



# Main

In [3]:
df = pd.read_csv("/home/labs/hornsteinlab/Collaboration/MOmaps/outputs/preprocessing/NIH/brenner/raw_metrics021224_all.csv")

In [4]:
df['Marker'].value_counts()

Marker
DAPI            32750
TUJ1            23825
ANAX11           3000
SNCA             3000
TDP43            3000
P54              3000
CD41             3000
KIF5A            3000
FMRP             3000
G3BP1            3000
SQSTM1           3000
MitoTracker      2975
NCL              2975
PSD95            2975
CLTC             2975
LAMP1            2975
FUS              2975
PURA             2975
Calreticulin     2975
TIA1             2975
NEMO             2950
DCP1A            2950
TOMM20           2950
Phalloidin       2950
GM130            2950
PML              2950
PEX14            2950
Name: count, dtype: int64

In [5]:
counts = df['Marker'].value_counts()
all_markers = df['Marker'].unique()
print(len(all_markers))
print(all_markers)

27
['ANAX11' 'DAPI' 'SNCA' 'TUJ1' 'DCP1A' 'NEMO' 'CLTC' 'PSD95' 'CD41' 'P54'
 'TDP43' 'GM130' 'Phalloidin' 'TOMM20' 'Calreticulin' 'LAMP1' 'FMRP'
 'SQSTM1' 'G3BP1' 'KIF5A' 'PEX14' 'PML' 'PURA' 'TIA1' 'FUS' 'MitoTracker'
 'NCL']


# Create Brenner report

In [6]:
max_samples = 3
metric_name = 'Target_Sharpness_Brenner'
percentiles_to_describe = np.arange(0, 1.001, 0.001)
percentile_ranges = [0, 0.1, 0.2, 0.3, 0.5, 0.7, 1, 2, 5, 10, 15, 20, 30, 40, 60, 75, 80, 85, 90, 95, 98, 99, 99.5, 99.7, 99.8,99.9,100]
output_folder = "brenner_reports"

In [7]:
# Ensure the folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for marker in all_markers:
    print(marker)
    df_marker = df.loc[df['Marker'] == marker]
    percentiles = df_marker[metric_name].describe(percentiles=percentiles_to_describe)

    pdf_path = os.path.join(output_folder, f'output_report_{marker}.pdf')
    with PdfPages(pdf_path) as pdf:    
        
        fig = plt.figure(figsize=(12, 8))  
        gs = GridSpec(2, 2, figure=fig, height_ratios=[4, 1])  
        fig.suptitle(f"Marker: {marker}", fontsize=16, fontweight='bold')
        
        # Histogram 1: Full Range
        ax1 = fig.add_subplot(gs[0, 0])
        create_histogram(df_marker[metric_name], percentiles, low_perc=1, high_perc=99)
        ax1.set_title("Histogram 1 - Full Range")
        
        # Histogram 2: Limited Range
        ax2 = fig.add_subplot(gs[0, 1])
        create_histogram(df_marker[metric_name], percentiles, low_perc=1, high_perc=99, x_min=1, x_max=99, overlay_Cellline=True)
        ax2.set_title("Histogram 2 - Limited Range")
        
        pdf.savefig(fig) 
        plt.close(fig) 

        for i in range(len(percentile_ranges) - 1):
            per_min = np.round(percentile_ranges[i],2)
            per_max = np.round(percentile_ranges[i + 1],2)
            text_output = f'Images between %{per_min} - {per_max}%\n'
            threshold = percentiles[f'{per_min}%']
            threshold_second = percentiles[f'{per_max}%']

            c = (df_marker[metric_name] >= threshold) & (df_marker[metric_name] <= threshold_second)

            df_marker_filtered = df_marker[c]
            df_marker_filtered = df_marker_filtered.sample(frac=1, random_state=1)

            text_output += (f"Number of {marker} images in threshold {threshold} "
                            f"({per_min}%) (and {threshold_second} ({per_max}%)): "
                            f"{len(df_marker_filtered)}\n\n")
            text_output += df_marker_filtered['CellLine'].value_counts().to_string() + "\n\n"
            text_output += df_marker_filtered['Condition'].value_counts().to_string() + "\n\n"

            fig = plt.figure(figsize=(12, 8))
            gs = GridSpec(3, 1, figure=fig, height_ratios=[1, 2, 0.1]) 
            text_ax = fig.add_subplot(gs[0, :]) 

            text_ax.axis('off') 
            text_ax.text(0.01, 0.99, text_output, ha='left', va='top', fontsize=12, wrap=True)

            filtered_paths = df_marker_filtered['Path'].values
            num_images = min(max_samples, len(filtered_paths))

            img_gs = gs[1].subgridspec(1, num_images, wspace=0.1)  
            for ind, path in enumerate(filtered_paths[:num_images]):
                # Read and process the image
                target_path = os.path.join(d, path)
                img = cv2.imread(target_path, cv2.IMREAD_ANYDEPTH)
                img = fit_image_shape(img, (1024, 1024))
                img = rescale_intensity(img)

                # Add image subplot
                ax = fig.add_subplot(img_gs[0, ind])
                ax.imshow(img, cmap='gray')
                put_tiles_grid(image=img, ax=ax)
                ax.axis('off')
                labels = show_label(path)
                perc_brenner = abs(percentiles[[per for per in percentiles.keys() if '%' in per]] - get_image_focus_quality(img)).idxmin()
                ax.set_title(f"{labels[1]}, {labels[3]}, {get_metrics(img, True)}, {perc_brenner}", color='purple', fontsize=10)

            # Save the current page
            plt.tight_layout()
            pdf.savefig(fig)
            plt.close(fig)  # Free memory

ANAX11
DAPI
SNCA
TUJ1
DCP1A
NEMO
CLTC
PSD95
CD41
P54
TDP43
GM130
Phalloidin
TOMM20
Calreticulin
LAMP1
FMRP
SQSTM1
G3BP1
KIF5A
PEX14
PML
PURA
TIA1
FUS
MitoTracker
NCL


-----------------------------------------------------